In [1]:
import torch
import nip

import os
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

DEVICE = 'cuda:0'

In [2]:
qmodel = qlib.QuantizedLlamaForCausalLM.from_pretrained(
	'/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/T256_L16_V2_K2_lbits10_LowBitSym_qtip_QAT_KDLoss_425steps',
    torch_dtype=torch.float16,
).to(DEVICE)

model_name = 'Llama2-7b-hf'
tokenizer = qlib.load_tokenizer(model_name)

LOL, Im custom!


In [3]:
dataloader = qlib.QATDataset(
    config=nip.load('/home/msst/repo/Quantization/configs/data/wikitext_test_seqlen4096.yaml'),
    tokenizer=tokenizer
).get_dataloader()


with torch.amp.autocast('cuda', dtype=torch.float16):
    res = qlib.evaluate(qmodel, dataloader, print_times=25)
    print(res)

 10%|▉         | 2/21 [00:28<04:25, 13.99s/it]

1.580849822059912e+106


 14%|█▍        | 3/21 [00:40<03:57, 13.21s/it]

1.0536837930249968e+106


 19%|█▉        | 4/21 [00:53<03:38, 12.84s/it]

...

1.108458313177767e+105


 52%|█████▏    | 11/21 [02:19<02:03, 12.32s/it]

1.6192084650329597e+105


 52%|█████▏    | 11/21 [02:24<02:11, 13.13s/it]


KeyboardInterrupt: 

In [4]:
# trellis_layer = qmodel.get_submodule('model.layers.7.self_attn.q_proj')
# test_input = torch.randn(4, 4096, 4096).half().to('cuda:0')

# %%timeit
# with torch.amp.autocast('cuda', dtype=torch.float16):
#     with torch.no_grad():
#         trellis_layer(test_input)

# full 48.8 ms
# wo incoh 44.8 ms
# wo linear 26.8 ms
# wo linear and incoh 22.8 ms
# full 1mad 61.6 ms
# full 1mad_short 73.3 ms
# full 1mad_mid 62.1 ms